In [2]:
import networkx as nx
import matplotlib as plt
import numpy as np
import pandas as pd
from itertools import permutations

In [3]:
def randomWave(wavelist):
    random_seed=np.random.randint(len(wavelist))
    return wavelist[random_seed]

In [4]:
def Packedwave(wave_DF):
    waveList=[]
    df=wave_DF.sort_values(by=["waveUsage"],ascending=[False]) 
    df.reset_index(drop=True, inplace=True)#reset the index sequence
    for i in range(len(df)):
        waveList.append(df["wave"][i])
        
    return waveList

def waveUpdate(Wave,Path,band):
    for i in range(band):
        wave_loc=Total_wave.index(Wave+i)
        wave_range[wave_loc]+=len(Path)
    #print(wave_range)
    #print(Total_wave)
    #print(randomseed)
    Wave_Infor={"wave":Total_wave,
               "waveUsage":wave_range,
               "random": randomseed}
    Wave_infor=pd.DataFrame(Wave_Infor)
    
    return Wave_infor
    

In [5]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges      

def pathBand(path):
    path_len=len(path)
    #print(path)
    #print(path[0][0])
    resource=Resource[path[0][0]][path[0][1]][path[0][2]]
    for i in range(1,path_len):
        resource=list(map(lambda x: x[0]+x[1], zip(resource, Resource[path[i][0]][path[i][1]][path[i][2]])))
    pathbandList=zero_runs(resource) 
    
    return pathbandList

In [6]:
def bypassType(path):
    path_len=len(path)
    if(path_len==1):
        bypass_type=0
    elif(path_len==2):
        edgeLoc0=edges_list.index((path[0][0],path[0][1]))
        edgeLoc1=edges_list.index((path[1][0],path[1][1]))
        if(path[1] in Out_bypassEdges[edgeLoc0][path[0][2]] and path[0] in In_bypassEdges[edgeLoc1][path[1][2]] ) :
            bypass_type=2
        else:
            bypass_type=0
            
    elif(path_len==3):       
        edgeLoc0=edges_list.index((path[0][0],path[0][1]))
        edgeLoc1=edges_list.index((path[1][0],path[1][1]))
        edgeLoc2=edges_list.index((path[2][0],path[2][1]))
        if(path[1] in Out_bypassEdges[edgeLoc0][path[0][2]] and path[0] in In_bypassEdges[edgeLoc1][path[1][2]] ) :
            bypass_type=1
        elif(path[2] in Out_bypassEdges[edgeLoc0][path[1][2]] and path[1] in In_bypassEdges[edgeLoc1][path[2][2]] ):
            bypass_type=1
        else:
            bypass_type=0
                       
            
    return bypass_type
                
            

In [7]:
def pathBand_Ocuupy(loc1,loc2,wave1,band,index):
    path=SD_pathLink[loc1][loc2]
    pathBand=ALL_SD_bandwidth[loc1][loc2]
    for i in range(len(path)):
        for j in range(band):
            #print(path)
            #print(wave1)
            #print(Resource[path[i][0]][path[i][1]][path[i][2]][wave1+j])
            Resource[path[i][0]][path[i][1]][path[i][2]][wave1+j]+=index+1
            
    return

        
def pathUpdate():
    SR_bandwidth=[]
    #MaxBand=[]
    Hops=[]
    for i in range(len(SD_pathLink)):
        band=[]
        #Max=[]
        hops=[]        
        for j in range(len(SD_pathLink[i])):
            #cost.append(pathCost(SD_pathLink[i][j]))
            hops.append(len(SD_pathLink[i][j]))
            b=pathBand(SD_pathLink[i][j])
            #Max.append(max(b[:,1] - b[:,0]))
            band.append(b)            
        #SR_pathLink_cost.append(cost)
        SR_bandwidth.append(band)
        #MaxBand.append(Max)
        Hops.append(hops)
    global ALL_SD_bandwidth
    ALL_SD_bandwidth=SR_bandwidth
        
    return

In [8]:
def CoreSetup():
    Nodes_list=list(G.nodes)
    #Link_bypass=[]
    for i in Nodes_list:
        neighborList=list(G.neighbors(i))
        #bypasscore_num=len(neighborList)-1# set the bypass core number within each fiber 
        for j in range(len(neighborList)):
            dis=1
            while(j+dis<len(neighborList)):
                input_loc=edges_list.index((neighborList[j],i))
                output_loc=edges_list.index((i,neighborList[j+dis]))
                successFlag=0
                for m in range(7):
                    if(len(bypassEdges[input_loc][m])==0):
                        for n in range(7):
                            if(len(bypassEdges[output_loc][n])==0):
                                bypassEdges[output_loc][n].append(((neighborList[j],i,m)))
                                bypassEdges[input_loc][m].append(((i,neighborList[j+dis],n)))
                                successFlag=1
                                break
                        if(successFlag==1):
                            break
                input_loc=edges_list.index((neighborList[j+dis],i))
                output_loc=edges_list.index((i,neighborList[j]))
                successFlag=0
                for m in range(7):
                    if(len(bypassEdges[input_loc][m])==0):
                        for n in range(7):
                            if(len(bypassEdges[output_loc][n])==0):
                                bypassEdges[output_loc][n].append(((neighborList[j+dis],i,m)))
                                bypassEdges[input_loc][m].append(((i,neighborList[j],n)))
                                successFlag=1
                                break
                        if(successFlag==1):
                            break
                                                                                                        
                dis+=1
    return;

In [9]:
def edgeToLinks(path):
    pathlist=[]
    path_len=len(path)
    corePossib=7**path_len
    for i in range(corePossib):
        List=[]
        List.append((path[0]+(i%7,)))
        edgeLoc=edges_list.index(path[0])
        
        if(len(In_bypassEdges[edgeLoc][i%7])>0):
            List=[]
            continue
        for j in range(1,path_len):
            List.append((path[j]+((int(i/(7**j)%7)),)))
            edgeLoc0=edges_list.index(path[j-1])
            edgeLoc1=edges_list.index(path[j])
            if(len(Out_bypassEdges[edgeLoc0][List[j-1][2]])>0 and List[j] not in Out_bypassEdges[edgeLoc0][List[j-1][2]] ) :

                List=[]
                break
            if(len(In_bypassEdges[edgeLoc1][List[j][2]])>0 and List[j-1] not in In_bypassEdges[edgeLoc1][List[j][2]] ) :

                List=[]
                break
        edgeLoc=edges_list.index(path[path_len-1])
        if(len(Out_bypassEdges[edgeLoc][int(i/(7**(path_len-1)))])>0):            
            List=[]
            continue 
        if(len(List)>0):
            pathlist.append(List)    
    return pathlist

In [10]:
def PathAlgorithm_1(PathList,hopsList,bandwidth,pathBandList,bypassList,wave,list1,list2):
    Max=np.zeros(len(pathBandList))
    Sum=np.zeros(len(pathBandList))
    for i in range(len(pathBandList)):
        #print (pathBandList[i][:,1] - pathBandList[i][:,0])
        #print(i)
        #print(PathList[0])
        Max[i]=max(pathBandList[i][:,1] - pathBandList[i][:,0])
        Sum[i]=sum(pathBandList[i][:,1] - pathBandList[i][:,0])
    
    infor={"place":range(len(PathList)),
           "bypass":bypassList,
           "bandwidth": Max,
           "hops": hopsList,
           "sum":Sum
         }
    
    Path_infor=pd.DataFrame(infor)     
    #print(Path_infor)
    df=Path_infor.sort_values(by=list1,ascending=list2) 
    df.reset_index(drop=True, inplace=True)#reset the index sequence
    #print(df)
    Path=[]
    
    for i in range(len(df)):
        successful_flag=0
        if(df["bandwidth"][i]>bandwidth):
            for j in pathBandList[df["place"][i]]:
                if(wave>=j[0] and wave+bandwidth<=j[1]):
                    Path=PathList[df["place"][i]]
                    traffic_feedback.append(df["bypass"][i])
                    successful_flag=1
                    break 
        if(successful_flag==1):
            break
    if(len(Path)==0):
        traffic_feedback.append(-1)
        #print(df)
    return Path


In [11]:
def PathOccupy_percent():
    Bypass_2=[]
    Bypass_1=[]
    Bypass_0=[]
    for i in range(len(edges_list)):
        if(edges_list[i] in Traffic_SD_pairs):
            for j in range(len((SD_pathLink[i]))):
                percent=1-(sum(ALL_SD_bandwidth[i][j][:,1]-ALL_SD_bandwidth[i][j][:,0])-1)/160
                if(Bypass_Type[i][j]==2):
                    Bypass_2.append((percent))
                elif(Bypass_Type[i][j]==1):
                    Bypass_1.append((percent))
                elif(Bypass_Type[i][j]==0):
                    Bypass_0.append((percent))
    Bypass=[]
    Bypass.append(Bypass_0)
    Bypass.append(Bypass_1)
    
    Bypass.append(Bypass_2)
    
    return Bypass

In [12]:
def Link_percentage():
    Edge=[]
    Link=[]
    for i in edges_list:
        Edge.append(str(i))
    for j in range(7):        
        occupy=[]
        for i in edges_list:
            #print(i)
            #print(list(np.nonzero(Resource[i[0]][i[1]][j])))
            k=np.array(Resource[i[0]][i[1]][j])
            per=np.count_nonzero(k)/160
            occupy.append(per)
        Link.append(occupy)
    df=pd.DataFrame(Link,columns=Edge)
    #print(df)
    #df.rename()
    return df
        

In [13]:
def Path_percent():
    Bypass_2=[]
    Bypass_1=[]
    Bypass_0=[]
    for i in range(len(edges_list)):
        if(edges_list[i] in Traffic_SD_pairs):
            for j in range(len((SD_pathLink[i]))):
                #percent=1-(sum(ALL_SD_bandwidth[i][j][:,1]-ALL_SD_bandwidth[i][j][:,0])-1)/160
                if(Bypass_Type[i][j]==2):
                    Bypass_2.append((SD_pathLink[i][j]))
                elif(Bypass_Type[i][j]==1):
                    Bypass_1.append((SD_pathLink[i][j]))
                elif(Bypass_Type[i][j]==0):
                    Bypass_0.append((SD_pathLink[i][j]))
    Bypass=[]
    Bypass.append(Bypass_0)
    Bypass.append(Bypass_1)
    
    Bypass.append(Bypass_2)
    
    return Bypass

In [14]:
TrafficCost=0
node_list=[0,1,2,3]
Traffic_SD_pairs=[(0,3),(0,1),(0,2),(1,2),(2,3),(1,3),(2,1)]
SR_pair=[(0,1),(1,0),(1,2),(2,1),(0,2),(2,0),(2,3),(3,2),(0,3),(3,0),(1,3),(3,1)]
edges_list=[(0,1),(1,0),(1,2),(2,1),(0,2),(2,0),(2,3),(3,2),(1,3),(3,1)]
bypassEdges=[]
In_bypassEdges=[]
Out_bypassEdges=[]
for i in range(len(edges_list)):
    BY=[]
    BY0=[]
    BY1=[]
    
    for j in range(7):
        BY.append([])
        BY0.append([])
        BY1.append([])
    bypassEdges.append(BY)
    In_bypassEdges.append(BY0)
    Out_bypassEdges.append(BY1)
    

   
#先对D=2的节点进行bypass设置，在对D=3的节点进行bypass设置
#每次设置都在该link中不存在

G=nx.DiGraph()
G.add_nodes_from(node_list)
G.add_edges_from(edges_list)
nx.draw(G)

#plt.show()


link_list=[] 

for k in range(len(edges_list)):
    link_list0=[]
    for i in range(7):
        link_list0.append(edges_list[k]+(i,)) 
    link_list.append(link_list0)

SD_path=[]
SR_pathLink=[]
for i in range(len(SR_pair)):
    path0=[]
    for path in nx.all_simple_paths(G, source=SR_pair[i][0], target=SR_pair[i][1]):
        path_edges=[]            
        for j in range(len(path)-1):
            path_edges.append((path[j],path[j+1]))
        path0.append(path_edges)
    SD_path.append(path0)


#CoreSetup() 
for i in range(len(bypassEdges)):
    Link=bypassEdges[i]
    for j in range(7):
        if(len(Link[j])>0):
            if(Link[j][0][0]==edges_list[i][1]):
                
                Out_bypassEdges[i][j]=Link[j]
            elif(Link[j][0][1]==edges_list[i][0]):
                In_bypassEdges[i][j]=Link[j]
                

In [15]:
SD_pathLink=[]  
for i in range(len(SD_path)):
    Links=[]
    for j in range(len(SD_path[i])):
        Links=Links+edgeToLinks(SD_path[i][j])
        
    SD_pathLink.append(Links)


In [16]:
Tag_range = list(permutations(range(3), 3))
Tag_list=["hops","bypass","sum"]
FalseorTrue=[]
for i in range(8):
    k1=int(i%2)
    k2=int((i%4)/2)
    k3=int(i/4)
    FalseorTrue.append((k1,k2,k3))
    
print(FalseorTrue)

FalseTrueList=[False,True]


[(0, 0, 0), (1, 0, 0), (0, 1, 0), (1, 1, 0), (0, 0, 1), (1, 0, 1), (0, 1, 1), (1, 1, 1)]


In [62]:
Total_bandwidth=161
Resource=np.zeros([4,4,7,161]).tolist()
ALL_SD_bandwidth=[]
MaxBand=[]
Hops=[]
Bypass_Type=[]
for i in range(len(SD_pathLink)):
    Band=[]
    Max=[]
    Hhops=[]
    Bypass=[]
    for j in range(len(SD_pathLink[i])):
        Hhops.append(len(SD_pathLink[i][j]))
        b=pathBand(SD_pathLink[i][j])
        Max.append(max(b[:,1] - b[:,0]))
        Band.append(b)
        Bypass.append(bypassType(SD_pathLink[i][j]))
    ALL_SD_bandwidth.append(Band)
    MaxBand.append(Max)
    Hops.append(Hhops)
    Bypass_Type.append(Bypass)

    


In [63]:
Total_wave=list(range(Total_bandwidth-1))
wave_range=list(np.zeros(len(Total_wave)))
randomseed=[]
for i in range(len(Total_wave)):
    randomseed.append(np.random.randint(len(Total_wave)))
Wave_Infor={"wave":Total_wave,
           "waveUsage":wave_range,
           "random": randomseed}
Wave_infor=pd.DataFrame(Wave_Infor)


In [76]:
Success_list=[]
Tag_List=[]
True_List=[]
for tag_selection in range(1):
    for true_selection in range(1):
    
    
        list1=['hops', 'sum', 'bypass']
        list2=[True,False,False]
        
        Total_bandwidth=161
        Resource=np.zeros([4,4,7,161]).tolist()
        ALL_SD_bandwidth=[]
        MaxBand=[]
        Hops=[]
        Bypass_Type=[]
        for i in range(len(SD_pathLink)):
            Band=[]
            Max=[]
            Hhops=[]
            Bypass=[]
            for j in range(len(SD_pathLink[i])):
                Hhops.append(len(SD_pathLink[i][j]))
                b=pathBand(SD_pathLink[i][j])
                Max.append(max(b[:,1] - b[:,0]))
                Band.append(b)
                Bypass.append(bypassType(SD_pathLink[i][j]))
            ALL_SD_bandwidth.append(Band)
            MaxBand.append(Max)
            Hops.append(Hhops)
            Bypass_Type.append(Bypass)
        
        Total_wave=list(range(Total_bandwidth-1))
        wave_range=list(np.zeros(len(Total_wave)))
        randomseed=[]
        for i in range(len(Total_wave)):
            randomseed.append(np.random.randint(len(Total_wave)))
        Wave_Infor={"wave":Total_wave,
                   "waveUsage":wave_range,
                   "random": randomseed}
        Wave_infor=pd.DataFrame(Wave_Infor)

        
        
        # In[214]:
        
        
        Traffic=pd.read_csv('traffic.csv',sep=',',names=['S','D','band'])
        SuccessFlag=np.zeros(len(Traffic))
        BypassFlag=np.zeros(len(Traffic))
        traffic_wave=[]
        traffic_path=[]
        traffic_feedback=[]
        Path_band_usage=[]
        Traffic_num=len(Traffic)
        for i in range(Traffic_num):
            traffic=(Traffic['S'][i],Traffic['D'][i])
            band=Traffic['band'][i]
            traffic_Path=SD_pathLink[SR_pair.index(traffic)]
            traffic_Hops=Hops[SR_pair.index(traffic)]
            traffic_Band=ALL_SD_bandwidth[SR_pair.index(traffic)]
            traffic_Bypass=Bypass_Type[SR_pair.index(traffic)]
            wave=Packedwave(Wave_infor)
            for j in range(len(wave)):
                PathFlag=0
                select_wave=wave[j]
                print(select_wave)
                selectPath=PathAlgorithm_1(traffic_Path,traffic_Hops,band,traffic_Band,traffic_Bypass,select_wave,list1,list2)
                #print(selectPath)
                if(len(selectPath) and select_wave+band<=160):
                    Wave_infor=waveUpdate(select_wave,selectPath,band)
                    Pathloc=traffic_Path.index(selectPath)
                    pathBand_Ocuupy(SR_pair.index(traffic),Pathloc,select_wave,band,i)
                    PathFlag=1
                    traffic_wave.append(select_wave)
                    traffic_path.append(selectPath)            
                    pathUpdate()
                    if(PathFlag==1):
                        SuccessFlag[i]=1
                    break
                
        Success_list.append(SuccessFlag.sum()) 
        Tag_List.append(list1)
        True_List.append(list2)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
1
0
1
0
1
0
1
2
0
1
0
1
2
0
0
1
2
0
1
0
1
0
1
0
1
0
1
2
3
0
0
1
0
1
2
3
0
1
2
0
0
1
2
0
0
1
2
0
1
2
0
1
2
0
1
2
3
4
0
1
2
0
1
0
1
2
0
1
2
3
0
1
2
3
0
1
0
1
2
0
1
0
1
0
1
2
0
1
2
3
4
0
0
1
2
3
0
1
2
3
4
0
1
2
3
0
1
2
0
1
2
0
2
1
3
4
0
2
1
3
0
2
1
3
4
0
2
1
3
0
3
2
1
4
0
3
2
1
4
0
3
2
1
4
0
3
2
1
4
0
3
0
3
2
1
4
0
3
2
1
4
0
3
2
1
4
5
0
3
2
1
4
0
3
2
4
0
3
4
0
3
4
2
1
5
0
3
4
2
1
5
6
0
3
4
1
2
5
0
3
4
1
2
5
0
3
4
1
2
0
3
4
2
1
5
0
3
4
2
1
5
6
0
3
4
2
1
5
6
0
3
4
2
1
5
6
0
3
4
2
1
5
0
3
4
2
1
5
6
7
8
0
3
4
2
1
5
6
7
8
0
3
4
2
1
5
6
0
3
4
2
1
5
6
0
3
4
2
1
5
6
8
7
9
10
0
3
4
2
1
5
6
8
7
9
13
0
3
4
2
1
5
6
8
0
3
4
2
1
5
6
8
0
3
4
2
1
5
0
3
4
2
1
5
8
0
3
4
2
1
5
8
6
9
7
13
0
3
4
2
1
5
8
6
9
7
13
0
3
4
2
1
5
8
0
3
4
2
1
5
8
9
6
7
13
0
4
3
2
1
5
8
9
0
4
3
2
1
5
8
9
6
7
13
14
15
0
4
3
2
1
5
8
0
3
4
2
8
0
3
4
8
2
1
5
9
6
7
13
10
14
15
11
12
16
0
3
4
8
2
1
5
9
0
3
4
8
0
8
3
4
2
9
1

21
0
8
1
15
11
10
9
18
5
4
3
6
2
17
13
14
21
19
24
23
20
22
16
12
25
26
7
27
28
29
30
0
8
1
15
11
10
9
18
5
4
3
6
2
17
13
14
21
19
24
23
20
22
16
12
25
26
7
27
28
29
30
0
8
1
15
11
10
9
18
5
4
3
6
2
17
13
14
21
0
8
1
15
11
10
9
18
5
4
3
6
2
17
13
0
8
1
15
13
11
10
9
18
5
4
3
6
2
17
0
9
1
17
15
13
11
10
18
8
6
5
4
3
2
14
21
0
9
1
17
15
13
11
10
18
8
6
5
4
3
2
21
14
24
23
0
9
1
17
15
13
11
10
18
8
6
5
4
3
2
21
14
23
24
19
22
20
16
12
25
26
7
27
28
29
30
0
9
1
17
15
13
11
10
18
8
6
5
4
3
2
21
14
23
24
19
22
20
16
12
25
26
7
27
28
29
0
9
1
17
15
13
11
10
18
8
6
5
4
3
2
21
14
23
24
19
22
20
16
12
25
26
7
27
28
29
30
0
9
1
17
15
13
11
10
18
8
6
5
4
3
2
21
0
9
21
1
17
15
13
11
10
18
8
4
2
6
5
3
14
23
24
19
20
22
16
12
25
26
7
28
27
29
0
9
21
1
17
15
13
11
10
18
8
4
2
6
5
3
14
23
24
19
20
22
16
12
25
26
7
28
0
9
21
1
17
15
13
11
10
18
8
4
2
6
5
3
14
23
24
19
20
22
16
12
25
26
7
28
29
27
30
0
9
21
1
17
15
13
11
10
18
8
4
2
6
5
3
14
23
24
19
20
22
16
12
25
26
7
28
29
27
30
0
9
21
1
17
15
13
11
1

6
5
10
4
3
2
19
28
20
29
34
7
12
22
27
35
33
32
31
30
16
36
0
11
25
24
23
21
18
17
15
14
13
26
1
9
8
6
5
10
4
3
2
19
28
20
29
34
7
12
22
27
35
33
32
31
30
16
36
0
11
25
24
23
21
18
17
15
14
13
26
1
9
8
6
5
10
4
3
2
19
28
0
11
25
24
23
21
18
17
15
14
13
26
1
9
8
6
5
10
4
3
2
28
19
29
20
34
7
12
22
27
33
36
35
32
31
30
16
37
0
11
25
24
23
21
18
17
15
14
13
26
1
9
8
6
5
10
4
3
2
28
19
29
20
34
0
11
25
24
23
21
18
17
15
14
13
26
1
5
9
8
10
2
6
3
4
28
29
19
34
20
7
35
36
0
11
25
24
23
21
18
17
15
14
13
26
1
5
9
8
10
2
6
3
4
28
29
19
34
20
7
36
35
12
22
30
31
32
33
27
16
37
0
11
25
24
23
21
18
17
15
14
13
26
1
5
9
8
10
2
6
3
4
28
29
19
34
20
7
36
35
12
22
30
31
32
33
27
16
37
38
0
11
25
24
23
21
18
17
15
14
13
26
1
5
9
8
10
2
6
3
4
28
29
19
34
20
7
36
35
12
22
30
31
32
33
27
16
37
0
11
25
24
23
21
18
17
15
14
13
26
1
5
9
8
10
2
6
3
4
28
29
19
34
20
7
36
35
12
22
30
31
37
32
33
27
16
38
0
11
25
24
23
21
18
17
15
14
13
26
1
5
9
8
10
2
6
3
4
28
29
19
34
20
7
36
35
12
22
30
31
38
37
32
27
33
16


16
45
46
0
14
36
29
28
1
26
25
24
23
21
18
17
15
40
13
9
2
3
5
6
8
4
10
11
7
41
19
38
34
22
20
42
39
33
35
12
37
32
43
44
27
31
30
16
45
46
0
14
36
29
28
1
26
25
24
23
21
18
17
15
40
13
9
2
3
5
6
8
4
10
11
7
41
19
38
34
22
0
14
36
29
28
1
26
25
24
23
21
18
17
15
40
13
9
2
3
5
6
8
4
10
11
22
7
41
19
38
34
20
42
39
33
35
12
37
32
43
44
27
31
30
16
45
46
0
14
36
29
28
1
26
25
24
23
21
18
17
15
40
13
9
2
3
5
6
8
4
10
11
22
7
41
19
38
34
20
42
39
33
35
12
37
32
43
44
0
14
40
36
29
28
1
26
25
24
23
18
17
15
21
13
10
2
3
4
6
8
9
5
11
22
19
38
34
41
7
20
42
39
44
32
35
37
12
33
43
45
27
31
30
16
46
47
0
14
40
36
29
28
1
26
25
24
23
18
17
15
21
13
10
2
3
4
6
8
9
5
11
22
19
38
34
41
7
20
42
39
44
32
35
0
14
40
36
29
28
1
26
25
24
23
18
17
15
21
13
10
2
3
4
6
8
9
5
11
22
19
38
34
41
7
20
42
35
39
44
32
37
12
33
43
45
27
31
30
16
46
47
0
14
40
36
29
28
1
26
25
24
23
18
17
15
21
13
10
2
3
4
6
8
9
5
11
22
19
38
34
41
7
20
42
35
39
44
32
37
12
33
43
45
27
31
30
16
46
47
0
14
40
36
29
28
1
26
25
24
23

13
14
2
7
45
47
41
48
38
22
34
39
20
35
33
32
43
37
31
30
12
46
51
16
27
52
53
54
55
56
57
58
0
28
19
21
23
24
25
26
1
17
36
40
42
44
49
50
18
29
3
4
15
6
5
8
9
10
11
13
14
2
7
45
47
41
48
38
22
34
39
20
35
33
32
43
37
31
30
12
46
51
16
27
52
0
28
19
21
23
24
25
26
1
17
36
40
42
44
49
50
18
29
3
4
15
6
5
8
9
10
11
13
14
2
7
45
47
41
48
38
22
34
39
20
35
33
32
43
37
31
30
0
28
19
21
23
24
25
26
1
17
36
40
42
44
49
50
18
29
3
4
15
6
5
8
9
10
11
13
14
2
7
45
47
41
48
38
22
34
33
39
32
20
35
31
30
43
37
12
46
51
16
27
52
53
54
55
56
57
58
59
0
28
19
21
23
24
25
26
1
17
36
40
42
44
49
50
18
29
3
4
15
6
5
8
9
10
11
13
14
2
7
45
47
41
48
38
22
34
33
39
32
20
35
31
30
43
37
12
46
51
0
28
19
21
23
24
25
26
1
17
36
40
42
44
49
50
18
29
3
4
15
6
5
8
9
10
11
13
14
2
7
45
47
41
48
38
22
34
33
39
32
20
35
31
30
43
37
51
0
28
19
21
23
24
25
26
1
17
36
40
42
44
49
50
18
29
3
4
15
6
5
8
9
10
11
13
14
2
7
45
47
41
48
38
22
34
33
39
32
20
35
31
30
43
51
37
12
46
16
27
52
53
54
55
56
57
58
59
0
28
19
21
2

...

140
142
145
146
147
148
149
150
151
152
153
154
155
157
158
139
137
119
136
120
121
122
123
124
125
126
127
128
129
130
131
133
134
135
78
80
75
29
21
22
23
24
25
26
28
30
40
31
32
33
34
35
36
38
20
19
18
17
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
39
37
41
55
57
42
59
61
62
63
64
65
66
69
70
71
72
73
74
56
58
54
50
43
44
45
46
47
48
49
159
52
51
53
104
132
141
67
144
77
82
27
68
101
156
143
60
0
97
99
100
102
103
105
106
107
108
109
110
111
112
113
114
115
98
96
117
95
79
1
81
83
84
85
86
87
88
89
90
91
92
93
94
116
118
76
138
140
142
145
146
147
148
149
150
151
152
153
154
155
157
158
139
137
119
136
120
121
122
123
124
125
126
127
128
129
130
131
133
134
135
78
80
75
29
21
22
23
24
25
26
28
30
40
31
32
33
34
35
36
38
20
19
18
17
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
39
37
41
55
57
42
59
61
62
63
64
65
66
69
70
71
72
73
74
56
58
54
50
43
44
45
46
47
48
49
159
52
51
53
104
132
141
67
144
77
82
27
68
101
156
143
60
0
97
99
100
102
103
105
106
107
108
109
110
111
112
113
114
115
98
96
117
95

63
64
65
66
69
70
71
72
73
74
56
58
54
50
43
44
45
46
47
48
49
159
52
51
53
104
132
141
67
144
77
82
27
68
101
156
143
60
0
97
99
100
102
103
105
106
107
108
109
110
111
112
113
114
115
98
96
117
95
79
1
81
83
84
85
86
87
88
89
90
91
92
93
94
116
118
76
138
140
142
145
146
147
148
149
150
151
152
153
154
155
157
158
139
137
119
136
120
121
122
123
124
125
126
127
128
129
130
131
133
134
135
78
80
75
29
21
22
23
24
25
26
28
30
40
31
32
33
34
35
36
38
20
19
18
17
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
39
37
41
55
57
42
59
61
62
63
64
65
66
69
70
71
72
73
74
56
58
54
50
43
44
45
46
47
48
49
159
52
51
53
104
132
141
67
144
77
82
27
68
101
156
143
60
0
97
99
100
102
103
105
106
107
108
109
110
111
112
113
114
115
98
96
117
95
79
1
81
83
84
85
86
87
88
89
90
91
92
93
94
116
118
76
138
140
142
145
146
147
148
149
150
151
152
153
154
155
157
158
139
137
119
136
120
121
122
123
124
125
126
127
128
129
130
131
133
134
135
78
80
75
29
21
22
23
24
25
26
28
30
40
31
32
33
34
35
36
38
20
19
18
17
2
3


153
154
155
157
158
139
137
119
136
120
121
122
123
124
125
126
127
128
129
130
131
133
134
135
78
80
75
29
21
22
23
24
25
26
28
30
40
31
32
33
34
35
36
38
20
19
18
17
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
39
37
41
55
57
42
59
61
62
63
64
65
66
69
70
71
72
73
74
56
58
54
50
43
44
45
46
47
48
49
159
52
51
53
104
132
141
67
144
77
82
27
68
101
156
143
60
0
97
99
100
102
103
105
106
107
108
109
110
111
112
113
114
115
98
96
117
95
79
1
81
83
84
85
86
87
88
89
90
91
92
93
94
116
118
76
138
140
142
145
146
147
148
149
150
151
152
153
154
155
157
158
139
137
119
136
120
121
122
123
124
125
126
127
128
129
130
131
133
134
135
78
80
75
29
21
22
23
24
25
26
28
30
40
31
32
33
34
35
36
38
20
19
18
17
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
39
37
41
55
57
42
59
61
62
63
64
65
66
69
70
71
72
73
74
56
58
54
50
43
44
45
46
47
48
49
159
52
51
53
104
132
141
67
144
77
82
27
68
101
156
143
60
0
97
99
100
102
103
105
106
107
108
109
110
111
112
113
114
115
98
96
117
95
79
1
81
83
84
85
86
87
88
89
90
91
92
9

44
45
46
47
48
49
159
52
51
53
104
132
141
67
144
77
82
27
68
101
156
143
60
0
97
99
100
102
103
105
106
107
108
109
110
111
112
113
114
115
98
96
117
95
79
1
81
83
84
85
86
87
88
89
90
91
92
93
94
116
118
76
138
140
142
145
146
147
148
149
150
151
152
153
154
155
157
158
139
137
119
136
120
121
122
123
124
125
126
127
128
129
130
131
133
134
135
78
80
75
29
21
22
23
24
25
26
28
30
40
31
32
33
34
35
36
38
20
19
18
17
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
39
37
41
55
57
42
59
61
62
63
64
65
66
69
70
71
72
73
74
56
58
54
50
43
44
45
46
47
48
49
159
52
51
53
104
132
141
67
144
77
82
27
68
101
156
143
60
0
97
99
100
102
103
105
106
107
108
109
110
111
112
113
114
115
98
96
117
95
79
1
81
83
84
85
86
87
88
89
90
91
92
93
94
116
118
76
138
140
142
145
146
147
148
149
150
151
152
153
154
155
157
158
139
137
119
136
120
121
122
123
124
125
126
127
128
129
130
131
133
134
135
78
80
75
29
21
22
23
24
25
26
28
30
40
31
32
33
34
35
36
38
20
19
18
17
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
39
37
41
55


127
128
129
130
131
133
134
135
78
80
75
29
21
22
23
24
25
26
28
30
40
31
32
33
34
35
36
38
20
19
18
17
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
39
37
41
55
57
42
59
61
62
63
64
65
66
69
70
71
72
73
74
56
58
54
50
43
44
45
46
47
48
49
159
52
51
53
104
132
141
67
144
77
82
27
68
101
156
143
60
0
97
99
100
102
103
105
106
107
108
109
110
111
112
113
114
115
98
96
117
95
79
1
81
83
84
85
86
87
88
89
90
91
92
93
94
116
118
76
138
140
142
145
146
147
148
149
150
151
152
153
154
155
157
158
139
137
119
136
120
121
122
123
124
125
126
127
128
129
130
131
133
134
135
78
80
75
29
21
22
23
24
25
26
28
30
40
31
32
33
34
35
36
38
20
19
18
17
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
39
37
41
55
57
42
59
61
62
63
64
65
66
69
70
71
72
73
74
56
58
54
50
43
44
45
46
47
48
49
159
52
51
53
104
132
141
67
144
77
82
27
68
101
156
143
60
0
97
99
100
102
103
105
106
107
108
109
110
111
112
113
114
115
98
96
117
95
79
1
81
83
84
85
86
87
88
89
90
91
92
93
94
116
118
76
138
140
142
145
146
147
148
149
150
151
152
153


In [77]:
a=SuccessFlag.sum()
print(a)

1605.0


In [78]:
k=0
for i in traffic_path:
    k+=len(i)
print(k/len(traffic_path))

1.3096573208722742


In [79]:
k=0
for i in range(Traffic_num):
    if(SuccessFlag[i]==1):
        k+=Traffic['band'][i]
print(k)

8255


In [80]:
print(Link_percentage())

   (0, 1)  (1, 0)   (1, 2)  (2, 1)   (0, 2)  (2, 0)   (2, 3)   (3, 2)  \
0     1.0     1.0  1.00000     1.0  1.00000     1.0  0.99375  0.98750   
1     1.0     1.0  1.00000     1.0  1.00000     1.0  0.99375  0.99375   
2     1.0     1.0  0.98125     1.0  1.00000     1.0  0.99375  0.98750   
3     1.0     1.0  0.98125     1.0  0.99375     1.0  1.00000  1.00000   
4     1.0     1.0  1.00000     1.0  1.00000     1.0  1.00000  1.00000   
5     1.0     1.0  0.98125     1.0  1.00000     1.0  1.00000  1.00000   
6     1.0     1.0  1.00000     1.0  1.00000     1.0  1.00000  1.00000   

    (1, 3)  (3, 1)  
0  0.99375     1.0  
1  0.99375     1.0  
2  0.98125     1.0  
3  0.98750     1.0  
4  0.99375     1.0  
5  0.98750     1.0  
6  0.98750     1.0  


In [81]:
k=0
for i in range(Traffic_num):
    if(SuccessFlag[i]==1):
        k+=Traffic['band'][i]
print(k)

8255
